# Basic Q&A System

### Sentence Retrival

In [5]:
import math,numpy,json,re,nltk
import time,re,os.path,sys
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import defaultdict
from numpy import multiply
from math import sqrt
from nltk.tokenize import word_tokenize
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import os.path as path
from collections import OrderedDict
from nltk.stem.wordnet import WordNetLemmatizer as WNL
from sklearn.feature_extraction import DictVectorizer
from nltk import FreqDist, DictionaryProbDist
from operator import add
import csv, ast
from nltk.tag.stanford import StanfordPOSTagger

#### Load data from json file

In [31]:
t0 = time.time()
filename_ls = ['QA_dev.json']
dataset = []
train_path = path.abspath('data/QA_dev.json')

dataset = []
with open(train_path) as f:
    for line in f:
        dataset+=(json.loads(line))
print "Import Successful "
print "There are totally", len(dataset),'documents in this dataset'

Import Successful 
There are totally 40 documents in this dataset


#### Build modle and evaluate the result

In [32]:
stopwords = set(nltk.corpus.stopwords.words('english')) # wrap in a set() (see below)
stemmer = nltk.stem.PorterStemmer() 

def my_tokenizer(doc):
    terms = set()
    for token in nltk.word_tokenize(doc):
        if token not in stopwords: # 'in' and 'not in' operations are much faster over sets that lists
            terms.add(stemmer.stem(token.lower()))
    return list(terms)


class MostRelevantSentenceModel(object):
    def __init__(self, vectorizer, collection_matrix):
        self.vectorizer = vectorizer
        self.collection_matrix = collection_matrix
        feature_array = vectorizer.get_feature_names()
        self.features = dict()
        for index in range(len(feature_array)):
            term = feature_array[index]
            self.features[term] = index

    def predict(self, queies):
        predictions = [self.inverted_index_score(i) for i in  queies]
        return predictions

    def inverted_index_score(self, query_sent):
        """
        now we implement inverted index to handle query
        
        :param query_sent: 
        :return: 
        
        """
        query_words = my_tokenizer(query_sent)
        score = defaultdict(float)

        for w in query_words:
            try:
                col_i = self.features[w]
                inverted_ix = self.collection_matrix[:, col_i]
                for di in range(inverted_ix.shape[0]):
                    score[di] += inverted_ix[di, 0]
            except KeyError:
                pass

        index_score = sorted(score.items(), key=lambda (k, v): v, reverse=True)

        if index_score:
            top10_doc_index = [i[0] for i in index_score[:10]]
            return top10_doc_index
        else:
            print 'error occured' ,query_sent
            return -1, 0

In [33]:
def build_model_and_evaluate(model, query ,report=False):
    evaluate_row = []
    pred = model.predict(query)
    quest_index = 0
    for pred_index in pred:
        drow = dict()
        if report:
            print pred_index
        drow['question_ID'] = quest_index
        drow['prediction_ID'] = pred_index
        evaluate_row.append(drow)
        quest_index += 1
    return evaluate_row


# #build model for each document collaction

for document in dataset:
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=1, use_idf=True,norm='l1',stop_words=None, tokenizer=my_tokenizer)
    document_collections_sents = document['sentences']
    tfidf_matrix = tfidf_vectorizer.fit_transform(document_collections_sents)
    document['model'] = MostRelevantSentenceModel(vectorizer=tfidf_vectorizer,collection_matrix=tfidf_matrix)

#### Output with predtion and actual values

In [34]:
import csv
csv_file = open('data/evaluatin_dev_results.csv', mode='w',)
fieldnames = ['document_ID', 'question_ID','question','prediction_ID','prediction_sentence']
writer = csv.DictWriter(csv_file, fieldnames=fieldnames, )
writer.writeheader()

ddi = 0
for document in dataset:
    questions = [i['question'] for i in document['qa']]
    model = document['model']
    result_row = build_model_and_evaluate(model, questions)
    doc_sents = document['sentences']
    for r in result_row:
        r['document_ID'] = ddi
        r['question'] = questions[r['question_ID']].encode('utf-8')
        r['prediction_sentence'] = doc_sents[r['prediction_ID'][0]].encode('utf-8')
        writer.writerow(r)
    ddi += 1
print 'Running time count:', time.time() - t0

error occured Who was the runner up?
error occured Who was the runner up?
error occured Where did it open?
error occured What did the actof of milno do?
error occured What is an Etsudiantinas? 
error occured What is crosspicking?
Running time count: 198.904000044


### Entity Extraction

In [2]:
def json_load_byteified(file_handle):
    return _byteify(json.load(file_handle, object_hook=_byteify),ignore_dicts=True)

def _byteify(data, ignore_dicts = False):
    # if this is a unicode string, return its string representation
    if isinstance(data, unicode):
        return data.encode('utf-8')
    # if this is a list of values, return list of byteified values
    if isinstance(data, list):
        return [ _byteify(item, ignore_dicts=True) for item in data ]
    # if this is a dictionary, return dictionary of byteified keys and values
    # but only if we haven't already byteified it
    if isinstance(data, dict) and not ignore_dicts:
        return {
            _byteify(key, ignore_dicts=True): _byteify(value, ignore_dicts=True)
            for key, value in data.iteritems()
        }
    # if it's anything else, return it in its original form
    return data

In [8]:
import os
java_path = "C:\\Program Files\\Java\\jre1.8.0_131\\bin" # replace this
os.environ['JAVAHOME'] = java_path

In [33]:
with open("data/QA_dev.json") as json_file:
    json_data = json_load_byteified(json_file)
print 'import success'

import success


In [ ]:
cwd = os.getcwd()
st = StanfordNERTagger(cwd+'\data\english.all.3class.distsim.crf.ser.gz',cwd+'\data\stanford-ner.jar')

if not os.path.isfile("data/NERdev.json"):    
    start = time.time()
    progressT = len(json_data)    
    listOfDocument=[]
    i=0
    for jd in json_data:
        aList=[]
        aList.append(st.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x['question']))) for x in jd['qa']]))
        #remove the below file if running on test set
        aList.append(st.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x['answer']))) for x in jd['qa']]))
        aList.append(st.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x))) for x in jd['sentences']]))
        listOfDocument.append(aList)
        i+=1
        sys.stdout.write('\r')
        sys.stdout.write("%d%%" % (i*100/progressT))
        sys.stdout.flush()    
    for document in range(0,len(listOfDocument)):
        #change [2] to [1] if test set
        for sentence in range(0,len(listOfDocument[document][2])):
            for word in range(0,len(listOfDocument[document][2][sentence])):   
                listOfDocument[document][2][sentence][word]= (listOfDocument[document][2][sentence][word][0]
                                                              ,listOfDocument[document][2][sentence][word][1] if not listOfDocument[document][2][sentence][word][0].isdigit() else u'NUMBER')
    with open('data/NERdev.json', 'w') as outfile:
        json.dump(listOfDocument, outfile)
    end = time.time()
    print '\nTime spending:',end - start    
else:    
    print 'there is a file'
with open("data/NERdev.json") as json_file:
        json_dataNER = json_load_byteified(json_file)

7%

In [28]:
with open("data/QA_train.json") as json_file:
    json_data = json_load_byteified(json_file)
print 'import success'

import success


In [28]:
cwd = os.getcwd()
stPOS = StanfordPOSTagger(cwd+'\\data\\wsj-0-18-left3words-distsim.tagger',cwd+'\data\stanford-postagger.jar')

if not os.path.isfile("data/POSdev.json"):    
    start = time.time()
    progressT = len(json_data)    
    listOfDocument=[]
    i=0
    for jd in json_data:
        aList=[]        
        aList.append(stPOS.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x['question']))) for x in jd['qa']]))
        #remove the below file if running on test set
        aList.append(stPOS.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x['answer']))) for x in jd['qa']]))
        aList.append(stPOS.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x))) for x in jd['sentences']]))
        listOfDocument.append(aList)
        i+=1
        sys.stdout.write('\r')
        sys.stdout.write("%d%%" % (i*100/progressT))
        sys.stdout.flush()    
    with open('data/POSdev.json', 'w') as outfile:
        json.dump(listOfDocument, outfile)
    end = time.time()
    print '\nTime spending:',end - start    
else:    
    print 'there is a file'
with open("data/POSdev.json") as json_file:
        json_dataNER = json_load_byteified(json_file)

TypeError: list indices must be integers, not str

# Determine answer

In [13]:
def detectQuestion(argument):
    argument = tokenizeUnicode(argument.lower())
    if 'what' in argument and len(set(argument) & set(['year','time','rankings']))!=0:
        return 'NUMBER'
    elif 'who' in argument or 'whom' in argument or 'whose' in argument or \
            ('what' in argument and len(set(argument) & set(['name']))!=0):
        return 'PERSON'
    elif 'when' in argument:
        return 'NUMBER'
    elif 'where' in argument or 'what' in argument and len(set(argument) & set(['part']))!=0:
        return 'LOCATION'
    elif 'how' in argument and len(set(argument) & set(['many','much','long','far']))!=0:
        return 'NUMBER'        
    elif 'why' in argument:
        return 'O'
    elif 'which' in argument and len(set(argument) & set(['year',"years"]))!=0:
        return 'NUMBER'
    else:
        return 'O'

In [14]:
def tokenizeUnicode(aUnicode):
    return word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',aUnicode)))

In [17]:
with open("data/NERdev.json") as json_file:
    json_data = json_load_byteified(json_file)
with open("data/POSdev.json") as json_file:
    json_dataPOS = json_load_byteified(json_file)
with open("data/QA_dev.json") as json_file:
    json_dataOrg = json_load_byteified(json_file)
    
print 'import success'

import success


In [16]:
csv_file = 'evaluatin_test_results.csv'
i=0
answerSecondFilter = []
dictDoc={}
question={}
with open(csv_file) as csvfile:
    readCSV = csv.DictReader(csvfile, delimiter=',')
    for row in readCSV:        
        document_i = int(row['document_ID'])
        question_i = int(row['question_ID'])
        filteredlistOfNERSentence = []
        question_type = detectQuestion(row['question'])
        question[document_i,question_i] = row['question']
        for x in ast.literal_eval(row['prediction_ID']):
            listOfNERSentence = json_data[document_i][1][x]                      
            if question_type in [k[1] for k in listOfNERSentence]:
                filteredlistOfNERSentence.append(x)
                i+=1
        dictDoc[document_i,question_i,question_type]=filteredlistOfNERSentence        
print 'success',len(dictDoc)

success 8455


In [74]:
dictDoc2 = {}
dictDoc3 = {}
for i,j,qtype in dictDoc:    
    questionNNP = set([x[0] for x in json_dataPOS[i][0][j] if x[1] == 'NNP'])
    maxCommon = 0
    maxSentence = ''
    for x in dictDoc[i,j,qtype]:
        answerNNP = [y[0] for y in json_dataPOS[i][1][x] if y[1] == 'NNP']
        compareIntersection = len(set(questionNNP)&set(answerNNP))
        if compareIntersection > maxCommon:
            maxCommon = compareIntersection
            maxSentence = json_dataPOS[i][1][x], json_data[i][1][x]
    if len(dictDoc[i,j,qtype]) != 0:
        if len(maxSentence) == 0:
            maxSentence = json_dataPOS[i][1][dictDoc[i,j,qtype][0]], json_data[i][1][dictDoc[i,j,qtype][0]]
        if qtype != 'O':
            dictDoc2[i,j]= ' '.join([k[0] for k in maxSentence[1] if k[1]==qtype])
            dictDoc3[i,j] = maxSentence[1]
        else:
            dictDoc2[i,j]= ' '.join([k[0] for k in maxSentence[0] if k[1]=='NNP'])
            dictDoc3[i,j] = maxSentence[0]
print 'success with answer'

success with answer


In [101]:
dictDoc2[0,5]

'Nicholas Russia Napoleon III'

In [116]:
import csv
csv_file = open('data/test_results.csv', mode='wb',)
fieldnames = ['id', 'answer',]
writer = csv.DictWriter(csv_file, fieldnames=fieldnames,delimiter=',')
writer.writeheader()
k = 0
totalDoc=len(json_data)
for i in range(0, totalDoc):
    for j in range(0,len(json_data[i][0])):
        k+=1
        dictToCSV={}
        dictToCSV['id'] = k
        dictToCSV['answer'] = dictDoc2.get((i,j),'NOT SURE')
        writer.writerow(dictToCSV)    
print 'success',k    

success 8974


In [18]:
csv_file = 'data/evaluatin_dev_results.csv'
i=0
answerSecondFilter = []
dictDoc={}
question={}
with open(csv_file) as csvfile:
    readCSV = csv.DictReader(csvfile, delimiter=',')
    for row in readCSV:        
        document_i = int(row['document_ID'])
        question_i = int(row['question_ID'])
        filteredlistOfNERSentence = []
        question_type = detectQuestion(row['question'])
        question[document_i,question_i] = row['question']
        for x in ast.literal_eval(row['prediction_ID']):
            listOfNERSentence = json_data[document_i][2][x]                      
            if question_type in [k[1] for k in listOfNERSentence]:
                filteredlistOfNERSentence.append(x)
                i+=1
        dictDoc[document_i,question_i,question_type]=filteredlistOfNERSentence        
print 'success',len(dictDoc)
dictDoc2 = {}
dictDoc3 = {}
for i,j,qtype in dictDoc:    
    questionNNP = set([x[0] for x in json_dataPOS[i][0][j] if x[1] == 'NNP'])
    maxCommon = 0
    maxSentence = ''
    for x in dictDoc[i,j,qtype]:
        answerNNP = [y[0] for y in json_dataPOS[i][2][x] if y[1] == 'NNP']
        compareIntersection = len(set(questionNNP)&set(answerNNP))
        if compareIntersection > maxCommon:
            maxCommon = compareIntersection
            maxSentence = json_dataPOS[i][2][x], json_data[i][2][x]
    if len(dictDoc[i,j,qtype]) != 0:
        if len(maxSentence) == 0:
            maxSentence = json_dataPOS[i][2][dictDoc[i,j,qtype][0]], json_data[i][2][dictDoc[i,j,qtype][0]]
        if qtype != 'O':
            dictDoc2[i,j]= ' '.join([k[0] for k in maxSentence[1] if k[1]==qtype])
            dictDoc3[i,j] = maxSentence[1]
        else:
            dictDoc2[i,j]= ' '.join([k[0] for k in maxSentence[0] if k[1]=='NNP'])
            dictDoc3[i,j] = maxSentence[0]
print 'success with answer'

success 8455
success with answer


In [25]:
json_dataPOS[0][1][0]

[[['active', 'JJ'], ['near-infrared', 'JJ'], ['illumination', 'NN']],
 [['Infrared', 'JJ'], ['astronomy', 'NN']],
 [['red', 'JJ']],
 [['Infrared', 'JJ'], ['thermal-imaging', 'NN'], ['cameras', 'NNS']],
 [['scientific', 'JJ']],
 [['700', 'CD'], ['nm', 'NN']],
 [['near-IR', 'NN']],
 [['780', 'CD'], ['nm', 'NN']],
 [['1050', 'CD'], ['nm', 'NN']],
 [['Leaves', 'VBZ']],
 [['emissivity', 'NN']],
 [['hotter', 'JJR']],
 [['infrared', 'JJ'], ['cameras', 'NNS']],
 [['Infrared', 'JJ'], ['vibrational', 'JJ'], ['spectroscopy', 'NN']],
 [['it', 'PRP'],
  ['will', 'MD'],
  ['absorb', 'VB'],
  ['a', 'DT'],
  ['photon', 'NN'],
  ['that', 'WDT'],
  ['has', 'VBZ'],
  ['the', 'DT'],
  ['same', 'JJ'],
  ['frequency', 'NN']],
 [['the', 'DT'],
  ['frequencies', 'NNS'],
  ['of', 'IN'],
  ['infrared', 'JJ'],
  ['light', 'NN']],
 [['the', 'DT'], ['mid-infrared', 'JJ']],
 [['infrared', 'JJ'], ['filters', 'NNS']],
 [['Digital', 'NNP'], ['cameras', 'NNS']],
 [['T-ray', 'NN'], ['imaging', 'NN']],
 [['far-infrared',

In [22]:
import csv
csv_file = open('data/dev_results.csv', mode='wb',)
fieldnames = ['document_i','question_i','question', 'answer','actual','sentence_answer','sentence_actual']
writer = csv.DictWriter(csv_file, fieldnames=fieldnames,delimiter=',')
writer.writeheader()
k = 0
totalDoc=len(json_data)
for i in range(0, totalDoc):
    for j in range(0,len(json_data[i][0])):
        dictToCSV={}
        dictToCSV['document_i'] = i
        dictToCSV['question_i'] = j
        dictToCSV['question'] = question[i,j]
        dictToCSV['answer'] = dictDoc2.get((i,j),'NOT SURE')
        dictToCSV['actual'] = str(json_dataPOS[i][1][j])+' '+str(json_data[i][1][j])
        dictToCSV['sentence_answer'] = dictDoc3[i,j]
        dictToCSV['sentence_actual'] = json_dataOrg[i]['sentences'][json_dataOrg[i]['qa'][j]['answer_sentence']]
        writer.writerow(dictToCSV)    
print 'success',k  

IndexError: list index out of range